# Import Libraries

In [1]:
import os
from os import listdir
from os.path import isfile, join
import xml.etree.cElementTree as ET
import re
import numpy as np
import pandas as pd

import nltk
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import csv
import argparse
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_fscore_support, roc_auc_score
from sklearn import metrics, preprocessing


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Required Libraries

#Base and Cleaning 
import json
import requests
import pandas as pd
import numpy as np
import emoji
import regex
import re
import string
from collections import Counter

In [3]:
#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
import pyLDAvis.gensim
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

In [4]:
#Natural Language Processing (NLP)
import spacy
import gensim
from spacy.tokenizer import Tokenizer
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
import datapane as dp 

import pickle 

from pprint import pprint

C:\Users\Admin\anaconda3\lib\site-packages\plac_ext.py:806: DeprecationWarning: The asyncore module is deprecated and will be removed in Python 3.12. The recommended replacement is asyncio
  import asyncore
C:\Users\Admin\anaconda3\lib\site-packages\plac_ext.py:807: DeprecationWarning: The asynchat module is deprecated and will be removed in Python 3.12. The recommended replacement is asyncio
  import asynchat


In [5]:
pd.set_option('max_colwidth', 150)

In [6]:
tweets = pd.read_csv("largedata/tweets-6k.csv")
tweets.head(2)

,Tweets
0,Spoke at @AllHealthReform event on health care reform this morning. No live tweeting allowed. Did it really happen? @ddiamond True. You can cover ...
1,"Meaning achieved! Today, my desperate quest for meaning in life has brought me to Crystal Palace. #CRYSUN @SunderlandAFC https://t.co/ZaZaf2Lfig H..."


In [7]:
tweets = tweets["Tweets"].values.tolist()

In [8]:
tweets[0]

'Spoke at @AllHealthReform event on health care reform this morning. No live tweeting allowed. Did it really happen? @ddiamond True. You can cover \'more\' with \'more\' for \'less.\' But you don\'t want to be one of the \'less\' who get \'le… https://t.co/Ui54o2m5LD The hard part was not gawking at the Chrysler Building next door. My son built a 6 ft replica of it in our backyard. https://t.co/0aftXVUR3P @nicholas_bagley @onceuponA The other key question is which of these would hit Plan Year \'17 and which would hit Plan Year \'18. Must read from @nicholas_bagley and @onceuponA and must ask if the goal is orderly transition or disruption.\n\nhttps://t.co/PmAUOeMo6x @squelchedalot 11 generations all in VA in my family. Can\'t speak to all the branches though. #Obamacare Executive Order is not a specific action. It is a declaration of administrative war. Orderly transition or massive disruption? The President goes by. #InaugurationDay. https://t.co/RDQSx1VWq4 Motorcade in the parade. #I

In [9]:
tweets[1]

'Meaning achieved! Today, my desperate quest for meaning in life has brought me to Crystal Palace. #CRYSUN @SunderlandAFC https://t.co/ZaZaf2Lfig He\'ll be the judge of that. https://t.co/UerwkOSt1u It\'s bin day AND #transferdeadlineday. This won\'t happen again in our lifetime. #DrainTheSwamp https://t.co/NHSFjpkPMF @danthomascomedy Haha yes. It\'s not just impressive or anything. Genuinely enjoying it. Seems best not to question it as you\'re going along. @danthomascomedy Worth it.  Me and Chopping are in the 300s.  It\'s an astonishing book. Never read anything like it. @danthomascomedy That\'s a belter!  How\'s #DanReadsInfiniteJest going? @JasonArnopp @RetroGamer_Mag I thought she was going out with Wayne Of The Exploding Fist? @JasonArnopp @RetroGamer_Mag I loved Space Harriet.  How\'s she doing? That\'s why Jill Stein\'s in the new Adam Sandler movie. https://t.co/BR8FpK7xXb #Trump45 is the political equivalent of a "100% Unauthorised" boy band calendar I keep my Linkin Park Cr

In [10]:
len(tweets[0])

11441

# Tweets Preprocessing

In [11]:

replacement_patterns = [
    #match url (i.e: https://t.co/5tF5G9VKtq)
    (r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ''),

    #match user (i.e: @cerpintor )
    (r'@\w+', ''),

    #match hashtag (i.e: #WomensMarchOnWashington)
    (r'#\w+', ''),

    #Replace "&..." with ''
    (r'&\w+', '')
]


In [12]:
class RegexReplacer(object):
    def __init__(self, patterns = replacement_patterns):
        self.patterns = [(re.compile(regrex), repl) for (regrex, repl) in patterns]
        
    # Replace the words that match the patterns with replacement words
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s)
            return s

In [13]:
def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in emoji.distinct_emoji_list(text)]
    #emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

In [14]:
def lemmatization(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    
    doc = nlp(" ".join(text)) 
    return [token.lemma_ for token in doc if token.pos_ in allowed_postags]

In [15]:
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])
tknz = TweetTokenizer()
replacer = RegexReplacer()
# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']
# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)
# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)

punc = string.punctuation

In [16]:

# Build the bigram and trigram models
bigram = gensim.models.Phrases(tweets, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tweets], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [17]:

def normalize(doc):
    
    for i in range(len(doc)):
        
        doc[i] = give_emoji_free_text(doc[i])
        
        #Tokenize with replacement
        doc[i] = tknz.tokenize(replacer.replace(doc[i]))
        
        #Filter stopwords, punctuations, and lowercase
        doc[i] = [w.lower() for w in doc[i] if w not in punc and w not in ALL_STOP_WORDS]
    
        # Bigram
        doc[i] = bigram_mod[doc[i]]
        
        #lemmatizer = WordNetLemmatizer()
        
        doc[i] = lemmatization(doc[i], allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

        
        
        #concat
#         doc[i] = ' '.join(w for w in doc[i])
        
    return doc

In [18]:
 def is_emoji(string): 
     """Returns True if the string is an emoji.""" 
     return string in unicode_codes.EMOJI_DATA 

In [19]:
tweets = normalize(tweets)

In [20]:

tweets[0] 

['speak',
 'health',
 'care',
 'reform',
 'morning',
 'live',
 'tweeting',
 'allow',
 'happen',
 'true',
 'cover',
 'hard',
 'gawk',
 'door',
 'son',
 'build',
 'backyard',
 '@nicholas_bagley',
 'key',
 'question',
 'hit',
 'plan',
 'year',
 'hit',
 'plan',
 'year',
 'must',
 'read',
 'ask',
 'orderly',
 'transition',
 'disruption',
 'generation',
 'family',
 'can',
 'speak',
 'branch',
 'executive',
 'order',
 'specific',
 'action',
 'declaration',
 'administrative',
 'orderly',
 'transition',
 'massive',
 'disruption',
 'president',
 'go',
 'motorcade',
 'inaugurationday',
 'protester',
 'chant',
 'really',
 'motorcade',
 'head',
 '#',
 'inaugurationday',
 'protester',
 'many',
 'slice',
 '#',
 'inaugurationday',
 'read',
 'work',
 'solution',
 'trade',
 'off',
 'view',
 'chant',
 'singe',
 'scene',
 '#',
 'inaugurationday',
 'start',
 'spread',
 'news',
 'bring',
 'act',
 '#',
 'fda2017',
 'quick',
 'thought',
 'ask',
 'how',
 'view',
 'grind',
 'poster',
 'reference',
 'interview',

In [21]:
join_tweets = [','.join(t) for t in tweets]

In [22]:
#dp.Report(dp.Table(pd.DataFrame(join_tweets, columns=['Tweets']))).publish(name='processed_tweets')



In [23]:
pd.DataFrame(join_tweets, columns=['Tweets'])

,Tweets
0,"speak,health,care,reform,morning,live,tweeting,allow,happen,true,cover,hard,gawk,door,son,build,backyard,@nicholas_bagley,key,question,hit,plan,ye..."
1,"mean,achieve,today,desperate,quest,mean,life,bring,#,will,judge,day,#,happen,lifetime,#,impressive,genuinely,enjoy,seem,good,question,go,worth,cho..."
2,"holy,shit,cap,scare,second,@kimmismile,suck,find,pet,hour,kill,hope,go,beach,place,tit,light,people,ask,relationship,advice,life,advice,single,lif..."
3,"how,will,know,wanna,go,home,official,music,video,studio,night,know,love,love,original,mix,tune,heat,#,nowplaying,dish,good,music,heat,hear,rest,he..."
4,"come,#,smirk,future,crowd,number,viable,hope,improve,remember,weekend,use,rugby,fan,people,weekend,amazing,costume,unbearably,cute,@richiehardcore..."
...,...
5995,"opening,come,school,think,local,shut,connection,ion,p,live,#,ghosttown,hellogwalia,town,coolgardie,hard,believe,large,city,fight,aussenate,job,pla..."
5996,"score,only,repeat,goal,@killianwoods7,mention,find,new,sport,reek,terrible,news,mad,bellew,depth,know,go,murder,need,big,cunt,go,love,get,rinse,ga..."
5997,"try,chill,proud,moms,dad,great,job,happy,dude,place,bring,memory,sure,will,probably,year,c,definitely,head,west,when,think,good,time,waste,good,ti..."
5998,"year,cycle,coincidentally,work,peace,yr,upset,@themediatweet,sure,picture,would,mean,ban,enter,country,lavender,arrow,point,source,look,ice,age,ho..."


In [24]:
(dp.Table(pd.DataFrame(join_tweets, columns=['Tweets']).head(10)))

C:\Users\Admin\anaconda3\lib\site-packages\bokeh\core\property\primitive.py:37: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  bokeh_bool_types += (np.bool8,)


# LDA Model Building

In [25]:
tweets[0]

['speak',
 'health',
 'care',
 'reform',
 'morning',
 'live',
 'tweeting',
 'allow',
 'happen',
 'true',
 'cover',
 'hard',
 'gawk',
 'door',
 'son',
 'build',
 'backyard',
 '@nicholas_bagley',
 'key',
 'question',
 'hit',
 'plan',
 'year',
 'hit',
 'plan',
 'year',
 'must',
 'read',
 'ask',
 'orderly',
 'transition',
 'disruption',
 'generation',
 'family',
 'can',
 'speak',
 'branch',
 'executive',
 'order',
 'specific',
 'action',
 'declaration',
 'administrative',
 'orderly',
 'transition',
 'massive',
 'disruption',
 'president',
 'go',
 'motorcade',
 'inaugurationday',
 'protester',
 'chant',
 'really',
 'motorcade',
 'head',
 '#',
 'inaugurationday',
 'protester',
 'many',
 'slice',
 '#',
 'inaugurationday',
 'read',
 'work',
 'solution',
 'trade',
 'off',
 'view',
 'chant',
 'singe',
 'scene',
 '#',
 'inaugurationday',
 'start',
 'spread',
 'news',
 'bring',
 'act',
 '#',
 'fda2017',
 'quick',
 'thought',
 'ask',
 'how',
 'view',
 'grind',
 'poster',
 'reference',
 'interview',

# How to use LDA Model
Topic modeling involves counting words and grouping similar word patterns to describe topics within the data. If the model knows the word frequency, and which words often appear in the same document, it will discover patterns that can group different words together.

We start with converting a collection of words to a bag of words, which is a list of tuples (word_id, word_frequency). gensim.corpora.Dictionary is a great tool for this:

In [26]:

id2word = Dictionary(tweets)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in tweets]

# View
print(corpus[:1])

[[(0, 18), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 2), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 3), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 2), (50, 1), (51, 2), (52, 1), (53, 9), (54, 1), (55, 1), (56, 1), (57, 2), (58, 2), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 2), (68, 2), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 2), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 2), (85, 1), (86, 5), (87, 2), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 5), (97, 1), (98, 3), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 2), (106, 1), (107, 4), (108, 1), (109, 1), (110, 1

What do these tuples mean? Let’s convert them into human readable format to understand:



In [27]:
[[(id2word[i], freq) for i, freq in doc] for doc in corpus[:1]]

[[('#', 18),
  ("'d", 1),
  ('-', 1),
  ('@alanparson', 1),
  ('@johnsonphotog', 1),
  ('@jrovner', 2),
  ('@katmayne', 1),
  ('@mamurrayflemimg', 1),
  ('@national', 1),
  ('@nicholas_bagley', 1),
  ('@pharmacist', 1),
  ('@phrma', 1),
  ('@ruralhealth', 1),
  ('absolutely', 1),
  ('act', 2),
  ('action', 2),
  ('add', 2),
  ('address', 1),
  ('administrative', 1),
  ('affordable', 1),
  ('ago', 1),
  ('allow', 1),
  ('amazing', 1),
  ('ancor2016', 1),
  ('arrive', 2),
  ('ask', 3),
  ('audience', 1),
  ('audits', 1),
  ('authority', 1),
  ('av', 1),
  ('avoid', 1),
  ('away', 2),
  ('back', 1),
  ('backyard', 1),
  ('bal', 1),
  ('ball', 1),
  ('baseball', 1),
  ('beget', 2),
  ('begin', 1),
  ('behavior', 1),
  ('believe', 1),
  ('bid', 1),
  ('big', 2),
  ('branch', 1),
  ('brand', 1),
  ('break', 1),
  ('bring', 2),
  ('brother', 1),
  ('build', 1),
  ('call', 2),
  ('can', 1),
  ('cap', 2),
  ('car', 1),
  ('care', 9),
  ('cell', 1),
  ('certainly', 1),
  ('chair', 1),
  ('change

Now let’s build an LDA topic model. We will use gensim.models.ldamodel.LdaModel for this:

In [28]:
from gensim.models.ldamodel import LdaModel

In [29]:
# Build LDA model
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=10,
                   random_state=0,
                   chunksize=100,
                   alpha='auto',
                   per_word_topics=True)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.515*"#" + 0.009*"thank" + 0.008*"great" + 0.005*"how" + 0.005*"new" + '
  '0.004*"love" + 0.004*"marketing" + 0.004*"day" + 0.003*"win" + '
  '0.003*"business"'),
 (1,
  '0.007*"new" + 0.007*"change" + 0.006*"say" + 0.005*"news" + 0.005*"-" + '
  '0.005*"need" + 0.005*"how" + 0.005*"thank" + 0.005*"health" + '
  '0.005*"story"'),
 (2,
  '0.098*"follow" + 0.074*"check" + 0.042*"automatically" + 0.039*"people" + '
  '0.036*"unfollowe" + 0.035*"thank" + 0.035*"late" + 0.023*"follower" + '
  '0.020*"person" + 0.013*"giveaway"'),
 (3,
  '0.059*"auspol" + 0.018*"earn" + 0.006*"incorrect" + 0.005*"cave" + '
  '0.005*"level" + 0.005*"badge" + 0.004*"closure" + 0.004*"@turnbullmalcolm" '
  '+ 0.004*"explorer" + 0.003*"misinformation"'),
 (4,
  '0.019*"love" + 0.017*"go" + 0.016*"thank" + 0.015*"day" + 0.012*"good" + '
  '0.010*"know" + 0.010*"time" + 0.009*"happy" + 0.009*"watch" + 0.009*"feel"'),
 (5,
  '0.013*"new" + 0.009*"good" + 0.008*"thank" + 0.008*"year" + 0.008*"day" + '
  '0

In [30]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=tweets, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3314576662679539


# pyLDAvis

In [31]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.226341  0.056783       1        1  26.709133
7      0.236459 -0.074186       2        1  22.657723
4      0.221239 -0.013329       3        1  18.317656
1      0.158739 -0.051633       4        1  11.356554
9      0.176347 -0.156155       5        1  11.188408
0      0.068200  0.330335       6        1   7.515178
6     -0.109529 -0.045333       7        1   1.632855
2     -0.300787  0.003817       8        1   0.505056
3     -0.338671 -0.025597       9        1   0.114550
8     -0.338337 -0.024702      10        1   0.002887, topic_info=       Term           Freq          Total Category  logprob  loglift
0         #  132269.000000  132269.000000  Default  30.0000  30.0000
374   thank   22584.000000   22584.000000  Default  29.0000  29.0000
265  people   16457.000000   16457.000000  Default  28.0000  28.0000
669   today   16383.000000   16383.000000  Default  27.0000  27.0000
232    more    9403.000000    9403.000000  Default  26.0000  26.0000
..      ...            ...            ...      ...      ...      ...
582     man       0.000697    6631.657000  Topic10 -11.7925  -5.6153
402     way       0.000697    9367.595649  Topic10 -11.7925  -5.9607
161   great       0.000697   16783.463187  Topic10 -11.7926  -6.5439
240     new       0.000697   17823.409055  Topic10 -11.7926  -6.6040
265  people       0.000697   16457.183588  Topic10 -11.7927  -6.5243

[811 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
0          1  0.034316     #
0          2  0.010577     #
0          3  0.012066     #
0          4  0.001111     #
0          5  0.007848     #
...      ...       ...   ...
411        8  0.000057  year
4084       1  0.977866     °
4084       2  0.003104     °
4084       4  0.015522     °
18730      3  0.989090     •

[2051 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 8, 5, 2, 10, 1, 7, 3, 4, 9])

- Each bubble represents a topic. The larger the bubble, the higher percentage of the number of tweets in the corpus is about that topic.
- Blue bars represent the overall frequency of each word in the corpus. If no topic is selected, the blue bars of the most frequently used words will be displayed.
- Red bars give the estimated number of times a given term was generated by a given topic. As you can see from the image below, there are about 22,000 of the word ‘go’, and this term is used about 10,000 times within topic 1. The word with the longest red bar is the word that is used the most by the tweets belonging to that topic.
- The further the bubbles are away from each other, the more different they are. For example, it is difficult to tell the difference between topics 1 and 2. They seem to be both about social life, but it is much easier to tell the difference between topics 1 and 3. We can tell that topic 3 is about politics.

A good topic model will have big and non-overlapping bubbles scattered throughout the chart. As we can see from the graph, the bubbles are clustered within one place. Can we do better than this?

Yes, because luckily, there is a better model for topic modeling called LDA Mallet.

# How to use LDA Mallet Model

Our model will be better if the words in a topic are similar, so we will use topic coherence to evaluate our model. Topic coherence evaluates a single topic by measuring the degree of semantic similarity between high scoring words in the topic. A good model will generate topics with high topic coherence scores.

In [32]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=tweets, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('nCoherence Score: ', coherence_lda)


nCoherence Score:  0.3314576662679539


This is our baseline. We have just used Gensim’s inbuilt version of the LDA algorithm, but there is an LDA model that provides better quality of topics called the LDA Mallet Model.

# Let’s see if we can do better with LDA Mallet. 

In [33]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

from pprint import pprint

import spacy

import pickle
import re
import pyLDAvis
import pyLDAvis.gensim

import matplotlib.pyplot as plt
import pandas as pd

In [34]:
mallet_path = 'patt/to/mallet-2.0.8/bin/mallet' # update this path
#ldamallet = gensim.models.wrapper.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

gensim.models.wrapper.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)


AttributeError: module 'gensim.models' has no attribute 'wrapper'

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=tweets, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('nCoherence Score: ', coherence_ldamallet)